In [ ]:
% pylab inline 
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_csv("B:/Case studies/Student Hunt/train.csv")
test = pd.read_csv("B:/Case studies/Student Hunt/test.csv")

# VISUALIZATION

In [ ]:
# to check data
train.head()
test.head()

In [ ]:
# to understand the general statistics of data
train.describe()

In [42]:
# data type and dimension of features
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13152 entries, 0 to 13151
Data columns (total 10 columns):
ID             13152 non-null int64
Count          13152 non-null int64
ID1            13152 non-null object
Year           13152 non-null object
Month          13152 non-null object
Day            13152 non-null object
Time           13152 non-null object
date           13152 non-null datetime64[ns]
day-of-week    13152 non-null object
weekno         13152 non-null int64
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 1.0+ MB


In [ ]:
#  for checking null values
train.isnull().sum()

In [ ]:
train.boxplot(column='Count')

In [ ]:
train['Count'].hist(bins=10)

In [ ]:
Q1 = train['Count'].quantile(0.25)
Q3 = train['Count'].quantile(0.75)
IQR = Q3 - Q1

In [ ]:
mean_value = train.Count.mean()


lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

def imputer(value):
    if value < lower_limit or value > upper_limit:
        return mean_value
    else:
        return value

result = train.Count.apply(imputer)

In [ ]:
result.plot(kind='box')

#The above result is almost free from outliers, but it might have lost the important information. let not use this result right now.

In [ ]:
train['Count'].describe()

In [ ]:
train.ID.dtype

In [ ]:
# Splitting ID in to number of features
train['ID1'] = train['ID'].astype(str)
train['Year'] = train['ID1'].str[0:4]
train['Month'] = train['ID1'].str[4:6]
train['Day'] = train['ID1'].str[6:8] 
train['Time'] = train['ID1'].str[8:10] 

In [ ]:
# splitting ID int to no of feature
test['ID1'] = test['ID'].astype(str)
test['Year'] = test['ID1'].str[0:4]
test['Month'] = test['ID1'].str[4:6]
test['Day'] = test['ID1'].str[6:8] 
test['Time'] = test['ID1'].str[8:10] 

In [ ]:
train.head()
test.head()

In [ ]:
train.info()

In [ ]:
# New feature DATE
train['date'] = train['Year']+'-'+ train['Month'] + '-' + train['Day']
test['date'] = test['Year']+'-'+ test['Month'] + '-' + test['Day']

In [ ]:
# converting date in to datetime type
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [ ]:
# To determine day of week and week number
train['day-of-week'] = train['date'].dt.weekday_name
test['day-of-week'] = test['date'].dt.weekday_name
train['weekno'] = train['date'].dt.week
test['weekno'] = test['date'].dt.week

In [ ]:
train.head(100)

In [ ]:
train.boxplot(column = 'Count',by = 'day-of-week')

In [ ]:
train.shape

In [ ]:
temp = train.pivot_table(values = 'Count',index = 'day-of-week',columns='Month',aggfunc='sum')
temp

It can be seen From the above two outputs on saturday and sunday, The Number of count is much lower as compared to other week days.

In [ ]:
train['day-of-week'] = train['day-of-week'].str.replace('Saturday','6')
train['day-of-week'] = train['day-of-week'].str.replace('Sunday','6')
train['day-of-week'] = train['day-of-week'].str.replace('Monday','1')
train['day-of-week'] = train['day-of-week'].str.replace('Tuesday','2')
train['day-of-week'] = train['day-of-week'].str.replace('Wednesday','3')
train['day-of-week'] = train['day-of-week'].str.replace('Thursday','4')
train['day-of-week'] = train['day-of-week'].str.replace('Friday','5')

test['day-of-week'] = test['day-of-week'].str.replace('Saturday','6')
test['day-of-week'] = test['day-of-week'].str.replace('Sunday','6')
test['day-of-week'] = test['day-of-week'].str.replace('Monday','1')
test['day-of-week'] = test['day-of-week'].str.replace('Tuesday','2')
test['day-of-week'] = test['day-of-week'].str.replace('Wednesday','3')
test['day-of-week'] = test['day-of-week'].str.replace('Thursday','4')
test['day-of-week'] = test['day-of-week'].str.replace('Friday','5')

In [ ]:
train.boxplot(column = 'Count',by = 'Year')

In [43]:
train['day-of-week'].unique() = No

SyntaxError: can't assign to function call (<ipython-input-43-a4f7149735f9>, line 1)

In [37]:
No = ['6','7','1','2','3','4','5']

In [ ]:
train['Year'] = train['Year'].astype('int')
test['Year'] = test['Year'].astype('int')

In [ ]:
train['old'] = 2014 - train['Year']
test['old'] = 2014 - test['Year']

In [ ]:
ctrain=train[:11400]
ctest =train[11400:]
X = ctrain[['old','Month','Time','day-of-week','weekno']]
Y = ctrain[['Count']]
Xtest = ctest[['old','Month','Time','day-of-week','weekno']]
# importing random forest
from sklearn.ensemble import RandomForestRegressor
Regressor=RandomForestRegressor(n_estimators= 300 ,max_features=4, min_samples_leaf=5,max_depth = 12, oob_score=True)
Regressor.fit(X,Y)
predicted= Regressor.predict(Xtest)
predicted=np.around(predicted)
from sklearn.metrics import mean_squared_error
from math import sqrt
rms = sqrt(mean_squared_error(ctest['Count'], predicted))


In [ ]:
print(Regressor.feature_importances_)

In [ ]:
rms

In [ ]:
# Final Modelling
X = train[['old','Month','Time','day-of-week','weekno']]
Y = train[['Count']]
Xtest = test[['old','Month','Time','day-of-week','weekno']]
from sklearn.ensemble import RandomForestRegressor
Regressor=RandomForestRegressor(n_estimators= 300 ,max_features=4, min_samples_leaf=5,max_depth = 12,oob_score=True)
# fitting training dataset
Regressor.fit(X,Y)
# prediction
predicted2= Regressor.predict(Xtest)
predicted2

In [ ]:
print(Regressor.feature_importances_)

In [ ]:
predicted2=np.around(predicted2)

In [ ]:
# Make submission file and submit
submission = pd.DataFrame({'Count':predicted2,'ID':test['ID']},columns=['ID','Count'])
submission.to_csv('av15.csv', index = False)